In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "drive/MyDrive/nlp"

In [ ]:
!pip install -q -U trl transformers accelerate peft datasets bitsandbytes evaluate git+https://github.com/huggingface/huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig

model_name = "google-bert/bert-base-uncased"

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tjasad/Slovene_SuperGLUE_BoolQ")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
def preprocess_data(examples):

  passage_inputs = [f"passage : {x} " for x in examples["passage"]]
  question_inputs = [f"question : {x} " for x in examples["question"]]
  inputs = [passage_input + question_input for passage_input, question_input in zip(passage_inputs, question_inputs)]

  model_inputs = tokenizer(inputs, max_length=400, truncation=True)

  model_inputs["labels"] = examples["label"]
  model_inputs["labels"] = [int(label) for label in model_inputs["labels"]]

  return model_inputs

tokenized_dataset = dataset.map(preprocess_data, remove_columns=['idx', 'passage', 'label', 'question'], batched=True)

tokenized_train = tokenized_dataset["train"]
tokenized_val = tokenized_dataset["eval"]

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
id2label = {0: "False", 1: "True"}
label2id = {"False": 0, "True": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

model.config.use_cache = False

In [12]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

NUM_VIRTUAL_TOKENS = 12

peft_config = PromptTuningConfig(
    peft_type="PROMPT_TUNING",
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,
    num_layers=6,
    token_dim=768,
    num_attention_heads=12,
    tokenizer_name_or_path=model_name #The pre-trained model
)

In [ ]:
peft_model = get_peft_model(model, peft_config)
print(peft_model.print_trainable_parameters())

In [14]:
from transformers import TrainingArguments

new_model_name = "/prompt_fine_tuned_boolq_bert"
output_directory = "." + new_model_name

training_args = TrainingArguments(
    output_dir = output_directory,
    learning_rate= 2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    use_cpu=False,
    load_best_model_at_end=True,
    num_train_epochs=6,
)

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.684394,0.500000,0.533333
2,No log,0.670020,0.611111,0.640808
3,No log,0.657153,0.777778,0.777778
4,No log,0.649218,0.833333,0.791398
5,No log,0.645428,0.833333,0.791398
6,No log,0.644361,0.833333,0.791398


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=72, training_loss=0.6935620837741427, metrics={'train_runtime': 34.1211, 'train_samples_per_second': 16.178, 'train_steps_per_second': 2.11, 'total_flos': 101984087483616.0, 'train_loss': 0.6935620837741427, 'epoch': 6.0})

In [ ]:
# save model to hub
model_location = "tjasad" + new_model_name
trainer.push_to_hub(model_location)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6443609595298767,
 'eval_accuracy': 0.8333333333333334,
 'eval_f1': 0.7913978494623656,
 'eval_runtime': 0.5301,
 'eval_samples_per_second': 33.956,
 'eval_steps_per_second': 5.659,
 'epoch': 6.0}

In [ ]:
# Example
text="passage : Bankovec za 20 evrov – Zaenkrat obstaja le ena celotna serija evrskih bankovcev, vendar pa izhaja nova serija, ki bo podobna sedanji. Evropska centralna banka bo pravočasno naznanila, kdaj bodo bankovci iz prve serije izgubili status zakonitega plačilnega sredstva. question : Ali je bankovec za 20 evrov iz prve serije še vedno zakonito plačilno sredstvo?"

classifier = pipeline("sentiment-analysis", model='tjasad/prompt_fine_tuned_boolq')
classifier(text)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_0', 'score': 0.5154208540916443}]